In [1]:
from src.mazeData import *
from src.maze2025GraphClass import mazeGraph
from pyvis.network import Network
from src.PS_agentPrograms import *
from src.mazeProblemClass import MazeProblem
from src.agents import *
from src.naigationEnvironmentClass import MazeNavigationEnvironment
from src.Lab4Environment import *
import streamlit as st

In [2]:
def makeMaze(n):
  size = (n,n)
  proba_0 =0.25 # resulting array will have 25%? of zeros
  proba_Enemy =0.1 # resulting array will have 10%? of enemy
  arrMaze=np.random.choice([0, 1, 2], size=size, p=[proba_0, 1-proba_0-proba_Enemy, proba_Enemy] )
  return arrMaze

initState = (random.randint(0,6),random.randint(0,6))
goalState = (random.randint(0,6),random.randint(0,6))

mazeSize=7
mainMaze = makeMaze(mazeSize)
MazeCheck(mainMaze,initState,goalState)
mazeAvalActs=defineMazeAvailableActions(mainMaze)
maze1TM=makeMazeTransformationModel(mazeAvalActs)
mazeWorldGraph=mazeGraph(maze1TM, mazeStatesLocations(list(maze1TM.keys())))

In [3]:
mainMaze

array([[1, 0, 1, 0, 1, 1, 1],
       [1, 1, 1, 1, 0, 1, 1],
       [0, 1, 2, 2, 0, 2, 0],
       [2, 1, 1, 0, 2, 1, 0],
       [1, 1, 0, 1, 0, 1, 1],
       [1, 1, 1, 1, 0, 1, 1],
       [0, 1, 1, 0, 0, 1, 2]])

In [4]:
net_maze = Network(heading="Lab 4 Maze",
                bgcolor ="#242020",
                font_color = "white",
                height = "750px",
                width = "100%" 
)
nodeColors={
    "wall":"red",
    "path": "White",
    "Goal": "Green",
    "Start": "Yellow",
    "Enemy": "Orange"
}
nodeColorsList=[]

for node in mazeWorldGraph.origin.keys():
    if mainMaze[node[0],node[1]]==1:
        nodeColorsList.append(nodeColors["path"])
    elif mainMaze[node[0],node[1]]==0:
        nodeColorsList.append(nodeColors["wall"])
    else:
        nodeColorsList.append(nodeColors["Enemy"])

nodes=["-".join(str(item) for item in el) for el in mazeWorldGraph.origin.keys()]

x_coords = []
y_coords = []

for node in mazeWorldGraph.origin.keys():
    x,y=mazeWorldGraph.getLocation(node)
    x_coords.append(x)
    y_coords.append(y)

sizes=[10]*len(nodes)
net_maze.add_nodes(nodes, color=nodeColorsList, x=x_coords, y=y_coords, size=sizes, title=nodes)

for node in net_maze.nodes:
    node['label']=''

edge_weights = {(intTupleTostr(k), intTupleTostr(v2)) : k2 for k, v in mazeWorldGraph.origin.items() for k2, v2 in v.items()}

edges=[]

for node_source in mazeWorldGraph.nodes():
    for node_target, action in mazeWorldGraph.get(node_source).items():
        #node_target or node_source is a tuple -> convert to str
        if (intTupleTostr(node_source),intTupleTostr(node_target)) not in edges and (intTupleTostr(node_target), intTupleTostr(node_source)):
            net_maze.add_edge(intTupleTostr(node_source),intTupleTostr(node_target), label=edge_weights[(intTupleTostr(node_source),intTupleTostr(node_target))])
            edges.append((intTupleTostr(node_source),intTupleTostr(node_target)))

for node in net_maze.nodes:
    if node['id']==intTupleTostr(goalState):
        node['color']=nodeColors['Goal']
    elif node['id']==intTupleTostr(initState):
        node['color']=nodeColors['Start']


In [5]:
net_maze.toggle_physics(False)

In [6]:
mp1=MazeProblem(initState, goalState, mazeWorldGraph)

In [7]:
BFSAgent=BestFirstSearchAgentProgram()
DLS=IDSearchAgentProgram()

In [8]:
BFSmazeAgent1=ProblemSolvingMazeAgentBFS(initState, mazeWorldGraph, goalState)
DLSAgent1=ProblemSolvingMazeAgentIDS(initState, mazeWorldGraph, goalState)

In [9]:
#maze_Env1=Lab4NavEnvironment(mazeWorldGraph)

maze_Env1=MazeNavigationEnvironment(mazeWorldGraph, mainMaze)
#maze_Env2=MazeNavigationEnvironment(mazeWorldGraph, mainMaze)

In [10]:
maze_Env1.add_thing(BFSmazeAgent1)
#if it runs X times change a wall to a path

The <Node (1, 6)> is being pushed to frontier ...
The <Node (1, 6)> is being extracted from frontier ...
The child <Node (1, 5)> is being pushed to frontier ...
The child <Node (0, 6)> is being pushed to frontier ...
The <Node (0, 6)> is being extracted from frontier ...
The child <Node (0, 5)> is being pushed to frontier ...
The <Node (0, 5)> is being extracted from frontier ...
The child <Node (0, 4)> is being pushed to frontier ...
The <Node (0, 4)> is being extracted from frontier ...
The <Node (1, 5)> is being extracted from frontier ...
The child <Node (2, 5)> is being pushed to frontier ...
The <Node (2, 5)> is being extracted from frontier ...
The child <Node (3, 5)> is being pushed to frontier ...
The <Node (3, 5)> is being extracted from frontier ...
We have found our goal:  <Node (3, 5)>!
Solution (a sequence of actions) from the initial state to a goal: ['left', 'down', 'down']
The Agent in (1, 6) with performance 17


In [11]:
maze_Env1.add_thing(DLSAgent1)

Solution (a sequence of actions) from the initial state to a goal: ['left', 'down', 'down']
The Agent in (1, 6) with performance 17


In [12]:
maze_Env1.run()

step 1:
Agent decided to do left.
Agent decided to do left.
Agent in (1, 5) with performance = 17
Agent in (1, 5) with performance = 17
step 2:
Agent decided to do down.
Agent decided to do down.
Agent in (2, 5) with performance = 16
Agent in (2, 5) is under attack!
Agent in (2, 5) with performance = 16
Agent in (2, 5) is under attack!
step 3:
Agent decided to do down.
Agent decided to do down.
Agent in (3, 5) with performance = 14.4
Agent reached all goals
Agent in (3, 5) with performance = 14.4
Agent reached all goals


In [13]:
#maze_Env2.add_thing(DLSAgent1)

In [14]:
#maze_Env2.run()

In [15]:
if BFSmazeAgent1.performance > DLSAgent1.performance:
    print("Uniform Cost Search Wins!")
elif DLSAgent1.performance > BFSmazeAgent1.performance:
    print("Iterative DLS Wins!")
else:
    print("It's a Tie!")

It's a Tie!


In [16]:
net_maze.show("graphMaze1.html", notebook=False)

graphMaze1.html
